In [82]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pyCloudy as pc 
from itertools import chain

# Tell pyCloudy where your cloudy executable is:
pc.config.cloudy_exe = '/Users/alvis/Packages/c23.01/source/cloudy.exe'

In [87]:
def make_model(dir_, model_name, dens, ab_O):
    full_model_name = '{0}_{1:.0f}_{2:.2f}'.format(model_name, dens, ab_O)
    r_min = 5e16
    dist = 1.26
    Teff = 45000
    qH = 47.
    options = ('no molecules',
                'no level2 lines',
                'no fine opacities',
                'atom h-like levels small',
                'atom he-like levels small',
                'COSMIC RAY BACKGROUND',
                'element limit off -8',
                )
    emis_tab = ['H  1  4861.33A',
            'H  1  6562.81A',
            'Ca B  5875.64A',
            'N  2  6583.45A',
            'O  1  6300.30A',
            'O  2  3726.03A',
            'O  2  3728.81A',
            'O  3  5006.84A',
            'BLND  4363.00A',
            'S  2  6716.44A',
            'S  2  6730.82A',
            'Cl 3  5517.71A',
            'Cl 3  5537.87A',
            'O  1  63.1679m',
            'O  1  145.495m',
            'C  2  157.636m']
    abund = {'He' : -0.92, 'C' : -3.15, 'N' : -4.0, 'Ne' : -4.00, 
             'S' : -5.35, 'Ar' : -5.80, 'Fe' : -7.4, 'Cl' : -7.00}
    abund['O'] = ab_O
    # Defining the object that will manage the input file for Cloudy
    c_input = pc.CloudyInput('{0}{1}'.format(dir_, full_model_name))
    # Filling the object with the parameters
    # Defining the ionizing SED: Effective temperature and luminosity.
    # The lumi_unit is one of the Cloudy options, like "luminosity solar", "q(H)", "ionization parameter", etc... 
    c_input.set_BB(Teff = Teff, lumi_unit = 'q(h)', lumi_value = qH)
    # Defining the density. You may also use set_dlaw(parameters) if you have a density law defined in dense_fabden.cpp.
    c_input.set_cste_density(dens)
    # Defining the inner radius. A second parameter would be the outer radius (matter-bounded nebula).
    c_input.set_radius(np.log10(r_min))
    c_input.set_abund(ab_dict = abund, nograins = True)
    c_input.set_other(options)
    c_input.set_iterate() # (0) for no iteration, () for one iteration, (N) for N iterations.
    c_input.set_sphere() # () or (True) : sphere, or (False): open geometry.
    c_input.set_emis_tab(emis_tab)
    c_input.set_distance(dist, 'kpc')
    c_input.print_input(to_file = True, verbose = False)


In [91]:
class CloudyModel:
    """For making Cloudy .in files"""
    
    def __init__(self, linelist='LineList_HII_NJC.dat', wavelength_units='angstroms'):
        self.model = []
        self.linelist = linelist
        self.wavelength_units = wavelength_units
        
    def set_model_parameter(self, model_parameter):
        self.model.append(model_parameter)
    
    def delete_model_parameter(self, model_parameter):
        self.model = [item for item in self.model if item not in model_parameter]
        
    def add_grid(self, parameter, start, stop, step, initial_value=0):
        self.set_model_parameter(f'{parameter} {initial_value} vary')
        self.set_model_parameter(f'grid {start} to {stop} step {step}')
        
    def set_other(self, *args):
        for arg in args:
            self.set_model_parameter(f'{arg}')
        
    def set_sed(self, sed):
        self.sed = sed
        self.set_model_parameter(f'table SED "{sed}"')
        
    def set_star(self, sed, age, stellar_metallicity):
        """Note: this requires intensity/luminosity/ionization parameter to be set or else Cloudy will break"""
        self.sed = f'{sed}_{age}_{stellar_metallicity}'
        self.set_model_parameter(f'table star "{sed}" {age} {stellar_metallicity}')

    def set_hden(self, hden):
        """Cloudy will break if there is no hden set"""
        self.hden = hden
        self.set_model_parameter(f'hden {hden}')
    
    def set_geometry(self, geometry):
        self.geometry = geometry
        self.set_model_parameter(f'{geometry}')
        
    def set_abundances(self, abundances, grains='no grains'):
        self.abundances = abundances
        self.set_model_parameter(f'abundances {abundances} {grains}')
        
    def set_grains(self, grains):
        self.grains = grains
        self.set_model_parameter(f'grains {grains}')
        
    def set_metals_and_grains(self, gas_metallicity):
        self.gas_metallicity = gas_metallicity
        self.set_model_parameter(f'metals and grains {gas_metallicity}')
        
    def set_element_scale_factor(self, element, element_scale_factor):
        self.set_model_parameter(f'element scale factor {element} {element_scale_factor}')
        
    def save_overview(self):
        self.set_model_parameter(f'save overview ".ovr" last')
        
    def save_continuum(self):
        self.set_model_parameter(f'save continuum units {self.wavelength_units} ".con" last')
    
    def save_lines_emergent(self):
        self.set_model_parameter(f'save linelist column emergent absolute last units {self.wavelength_units} ".elin" "{self.linelist}"')
        
    def save_lines_intrinsic(self):
        self.set_model_parameter(f'save linelist column intrinsic absolute last units {self.wavelength_units} ".elin" "{self.linelist}"')
        
    def save_all(self):
        self.save_overview()
        self.save_continuum()
        self.save_lines_emergent()
        self.save_lines_intrinsic()
        
    def build_default_model(self, sed='NGC5548.sed', hden=2, abundances='gass10', grains='Orion', gas_metallicity=1.0):
        self.model = []
        self.set_sed(sed)
        self.set_hden(hden)
        self.set_abundances(abundances)
        self.set_grains(grains)
        self.set_metals_and_grains(gas_metallicity)
        self.add_grid('ionization parameter', -4, -1, 0.25)
        self.set_model_parameter('iterate_to_convergence')
        self.save_all() 
        
    def build_cleri_model(self, sed='NGC5548.sed', hden=2, abundances='gass10', grains='Orion', gas_metallicity=1.0, element_scale_factor_dict={}):
        self.model = []
        self.set_sed(sed)
        self.set_hden(hden)
        self.set_abundances(abundances)
        self.set_grains(grains)
        self.set_metals_and_grains(gas_metallicity)
        for element in element_scale_factor_dict.keys():
            self.set_element_scale_factor(element, element_scale_factor_dict.get(element))
        self.add_grid('ionization parameter', -4, -1, 0.25)
        self.set_model_parameter('iterate_to_convergence')
        self.save_all()   
        
    def build_template_model_bpass(self, sed="BPASSv2.2.1_imf135_300_burst_binary.ascii", age=1e7, stellar_metallicity=-1, 
                                  hden=2, abundances='gass10', grains='Orion', gas_metallicity=1.0, element_scale_factor_dict={}):
        self.model = []
        self.set_star(sed, age, stellar_metallicity)
        self.set_hden(hden)
        self.set_abundances(abundances)
        self.set_grains(grains)
        self.set_metals_and_grains(gas_metallicity)
        for element in element_scale_factor_dict.keys():
            self.set_element_scale_factor(element, element_scale_factor_dict.get(element))
        self.add_grid('ionization parameter', -4, -1, 0.25)
        self.set_model_parameter('iterate_to_convergence')
        self.save_all()   
        
    def make_cloudy_in_file(self, path='.', use_params=True, comment=None):
        if not use_params:
            np.savetxt(f'{comment}.in', self.model, fmt='%s')
            return 
        
        if comment:
            np.savetxt(f'{path}/{self.sed.partition(".")[0]}_hden{self.hden}_z{self.gas_metallicity}_{comment}.in', self.model, fmt='%s')
        else:
            np.savetxt(f'{path}/{self.sed.partition(".")[0]}_hden{self.hden}_z{self.gas_metallicity}.in', self.model, fmt='%s')

In [44]:
x = CloudyModel()
x.build_cleri_model(element_scale_factor_dict={'nitrogen':0.4,'oxygen':0.5})
x.make_cloudy_in_file()

In [28]:
d = {'nitrogen':0.4,
     'oxygen':0.5}

In [87]:
metallicities = [0.1, 0.2, 0.3]
for i, z in enumerate(metallicities):
    model = CloudyModel()
    model.build_cleri_model(gas_metallicity=z)
    model.make_cloudy_in_file()

In [86]:
bpass = CloudyModel()
bpass.build_cleri_model()
bpass.model

['table SED "NGC5548.sed"',
 'hden 2',
 'abundances gass10 no grains',
 'grains Orion',
 'metals and grains 1.0',
 'ionization parameter 0 vary',
 'grid -4 to -1 step 0.25',
 'iterate_to_convergence',
 'save overview ".ovr" last',
 'save continuum units angstroms ".con" last',
 'save linelist column emergent absolute last units angstroms ".elin" "LineList_HII_NJC.dat"',
 'save linelist column intrinsic absolute last units angstroms ".elin" "LineList_HII_NJC.dat"']

In [81]:
stars = CloudyModel()
stars.set_model_parameter('table star "BPASS2.2/BPASSv2.2.1_imf135_300_burst_binary.ascii" 2e8 -1.8')
stars.set_model_parameter('ionization parameter -2')
stars.set_model_parameter('hden 2')
stars.make_cloudy_in_file(use_params=False, comment='test_stars')

In [79]:
stars = CloudyModel()
stars.set_star("BPASS2.2/BPASSv2.2.1_imf135_300_burst_binary.ascii", 2e8, -1)
stars.set_hden(2)
stars.make_cloudy_in_file(use_params=False, comment='available')

['table star "BPASS2.2/BPASSv2.2.1_imf135_300_burst_binary.ascii" 200000000.0 -1',
 'hden 2']

In [90]:
stars = CloudyModel()
stars.build_template_model_star()
stars.model

['table star "BPASS2.2/BPASSv2.2.1_imf135_300_burst_binary.ascii" 10000000.0 -1',
 'hden 2',
 'abundances gass10 no grains',
 'grains Orion',
 'metals and grains 1.0',
 'ionization parameter 0 vary',
 'grid -4 to -1 step 0.25',
 'iterate_to_convergence',
 'save overview ".ovr" last',
 'save continuum units angstroms ".con" last',
 'save linelist column emergent absolute last units angstroms ".elin" "LineList_HII_NJC.dat"',
 'save linelist column intrinsic absolute last units angstroms ".elin" "LineList_HII_NJC.dat"']